In [9]:
#import networkx as nx
import csv
import osmnx as ox
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib;
import pandas as pd 
import utm
import matplotlib.pyplot as plt 
from shapely.geometry import LineString, mapping
import geopandas as gpd
from ipyleaflet import *
from ipyleaflet import Map, FullScreenControl, Polyline
from datetime import datetime, timedelta, timezone
import subprocess
import numpy as np
ox.config(use_cache=True, log_console=True)
%matplotlib inline
from shapely.geometry import Point
import pyproj, math

class GpsObs:

    def __init__(self, data):
         # Class attributes
        self.time = data[1]
        self.name = data[2]
        self.satNum = data[3]
        self.carrier1 = data[5] # carrier-phase cycle 1
        self.carrier2 = data[6] # carrier-phase cycle 2
        self.prng1 = data[7] # pseudorange freq 1
        self.prng2 = data[8] # pesudorange freq 2
        self.doppler1 = data[9] # doppler frequency [Hz]
        self.doppler2 = data[10] # doppler frequency [Hz]
        self.satPos = [0, 0, 0]
       # self.occupGrid = OccupancyGrid()            # Generated Occupancy Grid  

gps_trace_file_path = "../dataset/UrbanNav/Tokyo/Odaiba/results.pos.trace"


# Table of dictionaries containing multiple obs_set
obs_tab = []
with open(gps_trace_file_path, 'r') as f:
    reader = csv.reader(f, delimiter='\n')
    data_trace = list(reader)
    idx_obs_s = -1
    idx_obs_e = -1
    idx_nav_s = -1
    idx_nav_e = -1
    # Set of multiple GPS observations (per given time)
    obs_set = {}
    
    for i, line in enumerate(data_trace[:1000]):
        # Convert array to string
        line = line[0]
        # Split everything in the same way
        line = line.split('/')[-1]
        if '#OBSERVATIONS_START' in line:
            idx_obs_s = i
        if '#OBSERVATIONS_END' in line:
            idx_obs_e = i
            # Add obs_set to the table and reset its value
            obs_tab.append(obs_set)
            obs_set={}
        if '#SATPOSITION_START' in line:
            idx_nav_s = i
        if '#SATPOSITION_END' in line:
            idx_nav_e = i
        
        if i > idx_obs_s and ((i < idx_obs_e) or (idx_obs_e < idx_obs_s)):
            line = line.split()
            if line[4] == 'rcv2':  # rcv2 - stationary base receiver
                continue
            new_obs = GpsObs(line)
            obs_set[new_obs.satNum] = new_obs
            
        if i > idx_nav_s and ((i < idx_nav_e) or (idx_nav_e < idx_nav_s)):
            line = line.split()
            satNum = line[2]
            obs = obs_tab[-1]
            if(satNum in obs):
                obs[satNum].satPos = [float(line[3]), float(line[4]), float(line[5])]

out_path = '/home/kcwian/Workspace/catkin_gnss_2/src/my_g2o/build/gps.g2o'
with open(out_path, 'w') as f:

    # Initial vertex # number, pose, quaternion
    f.write('VERTEX_SE3:QUAT 0 0 0 0 0 0 0 1\n')
    v_idx = 0
    e_idx = -0
    for obs_set in obs_tab:
        for key, value in  obs_set.items():
            v_idx += 1
            if v_idx > 0:
                if key in ['10', '13', '55', '78', '108', '113'] :
                    print(key)
                    continue
                f.write('VERTEX_SE3:QUAT {} {} {} {} 0 0 0 1\n'.format(v_idx, value.satPos[0], value.satPos[1], value.satPos[2]))
                f.write('FIX {}\n'.format(v_idx))

        for key, value in  obs_set.items():
            e_idx += 1
            # Edge from, to, distance, noise
            noise = 1000
            if e_idx > 0:
                if key in ['10', '13', '55', '78', '108', '113'] :
                    print(key)
                    continue
                f.write('GPSEDGE 0 {} {} {}\n'.format(e_idx, value.prng1, 1.0/noise))

        for key, value in  obs_set.items():
            pass
        
        # Only first for test
        break;
        
g2o_result_path = '/home/kcwian/Workspace/catkin_gnss_2/src/my_g2o/build/g2o_result.txt'
subprocess.call("./MyGPS", shell=True, cwd="/home/kcwian/Workspace/catkin_gnss_2/src/my_g2o/build/")
with open(g2o_result_path, 'r') as f:
    res = f.readline().split()
    print(res)
    

ecef_gpd = gpd.points_from_xy([res[0]], [res[1]], z=[res[2]], crs="EPSG:4978")
lon_lat_gpd = ecef_gpd.to_crs(crs="EPSG:4326")
print(lon_lat_gpd[0].y, lon_lat_gpd[0].x, lon_lat_gpd[0].z )

gps_result_file_path = "../dataset/UrbanNav/Tokyo/Odaiba/results.pos"
point = [lon_lat_gpd[0].y, lon_lat_gpd[0].x]
m = Map(center=point, basemap=basemaps.OpenStreetMap.Mapnik, zoom=5)
marker = Marker(location=point, draggable=False)
rtklib_est_ecef = [-3963425.455 ,3350879.447 ,3694864.328]
rtklib_est_ecef_gpd = gpd.points_from_xy([rtklib_est_ecef[0]], [rtklib_est_ecef[1]], z=[rtklib_est_ecef[2]], crs="EPSG:4978")
rtklib_est_lonlat = rtklib_est_ecef_gpd.to_crs(crs="EPSG:4326")
marker2 = Marker(location=(rtklib_est_lonlat[0].y, rtklib_est_lonlat[0].x), draggable=False)
m.add_layer(marker)
m.add_layer(marker2)
m

10
13
55
78
108
113
10
13
55
78
108
113
['-3503065.24288353', '3417124.2247421', '4264558.84453898']
41.257314885831484 135.7115129621023 122241.09000212513


Map(center=[41.257314885831484, 135.7115129621023], controls=(ZoomControl(options=['position', 'zoom_in_text',…